# Interface for Pipeline Execution
## (EUR-USD Exchange Data)

In [1]:
# Import models, metrics, paths and functions
from utils.helpers import csv_reader, display_ranking_table
from utils.paths import *
from pipeline.run_pipeline import run_pipeline

from models import MODELS
from metrics import METRICS

Loading paths...
Loading helper functions...
Loading data transformers...
Loading models...
Loading metrics...


In [2]:
# Get current models and metrics
# print(METRICS)
# print(MODELS)

In [3]:
## Using the EUR-USD Exchange Rate data

# Read EUR-USD exchange data
df = csv_reader(TESTDATA_DIR, 'eurusd', columns=['datetime', 'bid_close'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 143713 entries, 2020-01-01 00:00:00 to 2024-02-06 00:00:00
Data columns (total 1 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   bid_close  101921 non-null  float64
dtypes: float64(1)
memory usage: 2.2+ MB


In [4]:
display(df)

,bid_close
datetime,
2020-01-01 00:00:00,NaN
2020-01-01 00:15:00,NaN
2020-01-01 00:30:00,NaN
2020-01-01 00:45:00,NaN
2020-01-01 01:00:00,NaN
...,...
2024-02-05 23:00:00,1.07419
2024-02-05 23:15:00,1.07423
2024-02-05 23:30:00,1.07420


In [5]:
# Define reasonable subset for testing:
subset = df.head(10000)
display(subset)

,bid_close
datetime,
2020-01-01 00:00:00,NaN
2020-01-01 00:15:00,NaN
2020-01-01 00:30:00,NaN
2020-01-01 00:45:00,NaN
2020-01-01 01:00:00,NaN
...,...
2020-04-14 02:45:00,1.09466
2020-04-14 03:00:00,1.09463
2020-04-14 03:15:00,1.09456


In [6]:
output_dict = (
    run_pipeline(
        df=subset, models=MODELS, metrics=METRICS,
        #start="2020-01-01", end="2020-03-31",
        agg_method='last', agg_freq='B',
        forecast_init_train=0.3, ensemble_init_train=0.3,
        select_forecaster='all',
        select_ensemblers='all',
        verbose=True,
        fh=1
    )
)

[2024-03-07 13:02] Starting  Pipeline...

== Pipeline Step 1: Data Preprocessing ==

Searching time information...
Dates found in 'index' column!
Inferred frequency: 15T
Data goes from 2020-01-01 00:00 to 2020-04-14 03:45, resulting in 10000 observations.

Aggregating data to frequency 'business day' using method 'last' and dropping NaNs...
...finished!
Data now has 74 observations.

Selecting target and covariates...
Target: bid_close
Covariates: None

Data Insights:
            bid_close
datetime             
2020-01-02    1.11702
2020-01-03    1.11599
2020-01-06    1.11948
2020-01-07    1.11530
2020-01-08    1.11124

[Time elapsed: 00s]


== Pipeline Step 2: Individual Models' Predictions ==

Splitting data for training of forecasters (train/test ratio: 30/70)...
Initial training set has 23 observations and goes from 2020-01-02 to 2020-02-03.

In an historical expanding window approach, there are 51 periods to be forecasted by the individual models: 2020-02-04 to 2020-04-14
Out-of-s

C:\Users\Paul\Desktop\Repositories\Time-Series-Ensembles\project\utils\helpers\data_preparation.py:155: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' instead
  aggregated_df = data.resample(agg_freq).apply(method)
C:\Users\Paul\Desktop\Repositories\Time-Series-Ensembles\project\utils\helpers\data_preparation.py:155: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  aggregated_df = data.resample(agg_freq).apply(method)


Performing out-of-sample predictions...
...finished!

Now generating 51 one-step ahead historical expanding window predictions from model: Naive (drift) (sktime)
Performing out-of-sample predictions...
...finished!

Now generating 51 one-step ahead historical expanding window predictions from model: AutoSARIMA (sktime)
Auto-fitting model. Refitting every 17th period.
...forecast 1 / 51
...forecast 13 / 51
...forecast 26 / 51
...forecast 39 / 51
Performing out-of-sample predictions...
...finished!

Now generating 51 one-step ahead historical expanding window predictions from model: Exponential Smoothing (sktime)
Performing out-of-sample predictions...
...finished!

Now generating 51 one-step ahead historical expanding window predictions from model: STL (sktime)
Performing out-of-sample predictions...
...finished!

Now generating 51 one-step ahead historical expanding window predictions from model: XGBoost (darts)
Now performing corresponding out-of-sample predictions...
...finished!

Sk

In [8]:
display_ranking_table(output_dict['metrics ranking'])


Model,RMSE,MAPE,sMAPE,RMSE Ranking,MAPE Ranking,sMAPE Ranking
Weighted Ensemble: Inverse Error Covariance,0.009,0.006,0.395,1,1,1
Naive,0.009,0.006,0.418,2,2,2
Exponential Smoothing,0.009,0.006,0.419,3,3,3
Naive (drift),0.009,0.006,0.427,4,4,4
Meta Ensemble: Linear Regression,0.010,0.007,0.441,5,5,5
XGBoost,0.010,0.007,0.473,6,6,6
Weighted Ensemble: Inverse RMSE,0.010,0.007,0.482,8,7,7
Weighted Ensemble: Inverse Variance,0.010,0.007,0.487,7,8,8
Weighted Ensemble: Simple,0.011,0.008,0.506,10,9,9
Meta Ensemble: RandomForest,0.010,0.008,0.517,9,10,10
